In [2]:
import math
import random
import numpy as np
from collections import defaultdict, OrderedDict
import pdb
from collections import Counter
from numpy import genfromtxt
import scipy.io
from scipy import stats

import random
import Ipynb_importer
from node import Node
import math
import random
import numpy as np
from collections import defaultdict, OrderedDict
import pdb


In [132]:
if __name__ == "__main__":
    # dataset = "titanic"
    dataset = "titanic"

    if dataset == "titanic":
        # Load titanic data       
        path_train = 'datasets/titanic/titanic_training.csv'
        data = genfromtxt(path_train, delimiter=',', dtype=None)
        path_test = 'datasets/titanic/titanic_testing_data.csv'
        test_data = genfromtxt(path_test, delimiter=',', dtype=None)
        y = data[1:, 0]  # label = survived
        class_names = ["Died", "Survived"]
        
        # TODO: preprocess titanic dataset
        # Notes: 
        # 1. Some data points are missing their labels
        # 2. Some features are not numerical but categorical
        # 3. Some values are missing for some features
        
    elif dataset == "spam":
        features = [
            "pain", "private", "bank", "money", "drug", "spam", "prescription",
            "creative", "height", "featured", "differ", "width", "other",
            "energy", "business", "message", "volumes", "revision", "path",
            "meter", "memo", "planning", "pleased", "record", "out",
            "semicolon", "dollar", "sharp", "exclamation", "parenthesis",
            "square_bracket", "ampersand"
        ]
        assert len(features) == 32

        # Load spam data
        path_train = 'datasets/spam-dataset/spam_data.mat'
        data = scipy.io.loadmat(path_train)
        X = data['training_data']
        y = np.squeeze(data['training_labels'])
        Z = data['test_data']
        class_names = ["Ham", "Spam"]
         
    else:
        raise NotImplementedError("Dataset %s not handled" % dataset)
    
    """
    TODO: train decision tree/random forest on different datasets and perform the tasks 
    in the problem
    """

C:\Users\zhuwe\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  
C:\Users\zhuwe\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  # Remove the CWD from sys.path while we load stuff.


In [173]:
import pandas as pd

In [174]:
train_df = pd.read_csv('datasets/titanic/titanic_training.csv')
test_df = pd.read_csv('datasets/titanic/titanic_testing_data.csv')
train_df.tail()

,survived,pclass,sex,age,sibsp,parch,ticket,fare,cabin,embarked
995,0.0,3.0,male,16.0,0.0,0.0,347074,7.7750,NaN,S
996,1.0,3.0,male,22.0,0.0,0.0,2658,7.2250,NaN,C
997,0.0,2.0,male,63.0,1.0,0.0,24065,26.0000,NaN,S
998,0.0,3.0,female,41.0,0.0,5.0,3101295,39.6875,NaN,S
999,0.0,2.0,male,34.0,1.0,0.0,226875,26.0000,NaN,S


In [175]:
#consider deleting the ticket and cabin with so many Nans and categories
train_df = train_df.drop(['ticket', 'cabin'], axis=1)
test_df = test_df.drop(['ticket', 'cabin'], axis=1)

In [176]:
#only one missing value in sex, delete that row
train_df['sex'] = train_df['sex'].fillna('hhh')
train_df[train_df['sex'] == 'hhh']

,survived,pclass,sex,age,sibsp,parch,fare,embarked
705,NaN,NaN,hhh,NaN,NaN,NaN,NaN,NaN


In [177]:
train_df = train_df.drop(705, axis=0)

In [178]:
test_df['sex'] = test_df['sex'].fillna('hhh')
test_df[test_df['sex'] == 'hhh']

,pclass,sex,age,sibsp,parch,fare,embarked


In [179]:
# convert categorical variable to numeric
train_df['sex'] = train_df['sex']. \
    map( {'female': 1, 'male': 0} ).astype(int)
test_df['sex'] = test_df['sex']. \
    map( {'female': 1, 'male': 0} ).astype(int)
train_df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0.0,3.0,0,NaN,0.0,0.0,8.0500,S
1,0.0,1.0,0,22.0,0.0,0.0,135.6333,C
2,0.0,2.0,0,23.0,0.0,0.0,15.0458,C
3,0.0,2.0,0,42.0,0.0,0.0,13.0000,S
4,0.0,3.0,0,20.0,0.0,0.0,9.8458,S


In [180]:
# guessing the age 
guess_ages = np.zeros((2,3),dtype=int) 
guess_ages
combine = [train_df, test_df]

In [181]:
for idx,dataset in enumerate(combine):  
    # method adds a counter to an iterable and returns it in a form of enumerate object.     
    if idx==0:
        print('Working on Training Data set\n')
    else:
        print('-'*35)
        print('Working on Test Data set\n')
    
    print('Guess values of age based on sex and pclass of the passenger...')
    for i in range(0, 2):
        for j in range(0,3):
            guess_df = dataset[(dataset['sex'] == i) \
                        &(dataset['pclass'] == j+1)]['age'].dropna()

            # Extract the median age for this group
            # (less sensitive) to outliers
            age_guess = guess_df.median()
          
            # Convert random age float to int
            guess_ages[i,j] = int(age_guess)
    
            
    print('Guess_Age table:\n',guess_ages)
    print ('\nAssigning age values to NAN age values in the dataset...')
    
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.age.isnull()) & (dataset.sex == i) \
                    & (dataset.pclass == j+1),'age'] = guess_ages[i,j]
                    

    dataset['age'] = dataset['age'].astype(int)
train_df.head()

Working on Training Data set

Guess values of age based on sex and pclass of the passenger...
Guess_Age table:
 [[42 30 25]
 [36 26 22]]

Assigning age values to NAN age values in the dataset...
-----------------------------------
Working on Test Data set

Guess values of age based on sex and pclass of the passenger...
Guess_Age table:
 [[41 28 25]
 [33 30 19]]

Assigning age values to NAN age values in the dataset...


,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0.0,3.0,0,25,0.0,0.0,8.0500,S
1,0.0,1.0,0,22,0.0,0.0,135.6333,C
2,0.0,2.0,0,23,0.0,0.0,15.0458,C
3,0.0,2.0,0,42,0.0,0.0,13.0000,S
4,0.0,3.0,0,20,0.0,0.0,9.8458,S


In [182]:
for dataset in combine:    
    dataset.loc[ dataset['age'] <= 16, 'age'] = 0
    dataset.loc[(dataset['age'] > 16) & (dataset['age'] <= 32), 'age'] = 1
    dataset.loc[(dataset['age'] > 32) & (dataset['age'] <= 48), 'age'] = 2
    dataset.loc[(dataset['age'] > 48) & (dataset['age'] <= 64), 'age'] = 3
    dataset.loc[ dataset['age'] > 64, 'age']=4
train_df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0.0,3.0,0,1,0.0,0.0,8.0500,S
1,0.0,1.0,0,1,0.0,0.0,135.6333,C
2,0.0,2.0,0,1,0.0,0.0,15.0458,C
3,0.0,2.0,0,2,0.0,0.0,13.0000,S
4,0.0,3.0,0,1,0.0,0.0,9.8458,S


In [183]:
for dataset in combine:
    binary_encoded = pd.get_dummies(dataset.embarked)
    newcols = binary_encoded.columns
    dataset[newcols] = binary_encoded
   
train_df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,C,Q,S
0,0.0,3.0,0,1,0.0,0.0,8.0500,S,0,0,1
1,0.0,1.0,0,1,0.0,0.0,135.6333,C,1,0,0
2,0.0,2.0,0,1,0.0,0.0,15.0458,C,1,0,0
3,0.0,2.0,0,2,0.0,0.0,13.0000,S,0,0,1
4,0.0,3.0,0,1,0.0,0.0,9.8458,S,0,0,1


In [184]:
# dealing with fare
test_df['fare'].fillna(test_df['fare'].dropna().median(), inplace=True)
train_df['fare'].fillna(train_df['fare'].dropna().median(), inplace=True)
test_df.tail

<bound method NDFrame.tail of      pclass  sex  age  sibsp  parch      fare embarked  C  Q  S
0       1.0    1    1    0.0    0.0   69.3000        C  1  0  0
1       1.0    1    2    0.0    1.0   57.9792        C  1  0  0
2       3.0    0    0    5.0    2.0   46.9000        S  0  0  1
3       3.0    0    1    0.0    0.0    7.8750        S  0  0  1
4       2.0    0    1    0.0    0.0   13.0000        S  0  0  1
5       3.0    1    0    1.0    1.0   15.2458        C  1  0  0
6       2.0    0    1    0.0    0.0   13.8625        C  1  0  0
7       2.0    0    1    0.0    0.0   13.0000        S  0  0  1
8       2.0    0    3    0.0    0.0    9.6875        Q  0  1  0
9       1.0    0    2    0.0    0.0   29.7000        C  1  0  0
10      1.0    1    2    0.0    0.0  227.5250        C  1  0  0
11      2.0    1    1    0.0    0.0   13.0000        S  0  0  1
12      3.0    1    1    1.0    0.0   14.4542        C  1  0  0
13      3.0    1    1    3.0    1.0   25.4667        S  0  0  1
14      1.

In [185]:
# getting the cut of the fare
train_df['fareBand'] = pd.qcut(train_df['fare'], 4)
train_df[['fareBand', 'survived']].groupby(['fareBand'], as_index=False).mean().sort_values(by='fareBand', ascending=True)

,fareBand,survived
0,"(-0.001, 7.896]",0.232824
1,"(7.896, 14.458]",0.280335
2,"(14.458, 31.91]",0.459677
3,"(31.91, 512.329]",0.576000


In [186]:
for dataset in combine:
    for i in range(len(dataset)):
        if i == 705:
            continue
        if dataset['fare'][i] <=  7.896:
            dataset['fare'][i] = 0         
        if dataset['fare'][i]>  7.896 and dataset['fare'][i] <=14.458:
            dataset['fare'][i] = 1
        if dataset['fare'][i] >  14.458 and dataset['fare'][i] <=31.91:
            dataset['fare'][i] = 2
        if dataset['fare'][i] > 31.91:
            dataset['fare'][i] = 3
train_df[['fare','fareBand']].head()

C:\Users\zhuwe\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\zhuwe\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\zhuwe\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # R

,fare,fareBand
0,1.0,"(7.896, 14.458]"
1,3.0,"(31.91, 512.329]"
2,2.0,"(14.458, 31.91]"
3,1.0,"(7.896, 14.458]"
4,1.0,"(7.896, 14.458]"


In [187]:
train_df['fare'][999] = 2

C:\Users\zhuwe\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [189]:
train_df.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,C,Q,S,fareBand
995,0.0,3.0,0,0,0.0,0.0,0.0,S,0,0,1,"(-0.001, 7.896]"
996,1.0,3.0,0,1,0.0,0.0,0.0,C,1,0,0,"(-0.001, 7.896]"
997,0.0,2.0,0,3,1.0,0.0,2.0,S,0,0,1,"(14.458, 31.91]"
998,0.0,3.0,1,2,0.0,5.0,3.0,S,0,0,1,"(31.91, 512.329]"
999,0.0,2.0,0,2,1.0,0.0,2.0,S,0,0,1,"(14.458, 31.91]"


In [204]:
train_df = train_df.drop(columns='fareBand')
test_df = test_df.drop(columns='fareBand')

KeyError: "['fareBand'] not found in axis"

In [205]:
train_df = train_df.drop(columns='embarked')
test_df = test_df.drop(columns='embarked')

In [206]:
test_df.tail()

,pclass,sex,age,sibsp,parch,fare,C,Q,S
305,2.0,0,1,0.0,0.0,2.0,1,0,0
306,2.0,1,1,0.0,0.0,1.0,0,0,1
307,1.0,0,2,0.0,0.0,2.0,0,0,1
308,3.0,0,1,0.0,0.0,0.0,0,1,0
309,2.0,0,2,0.0,0.0,1.0,0,1,0


In [207]:
for dataset in combine:
    binary_encoded = pd.get_dummies(dataset.embarked)
    newcols = binary_encoded.columns
    dataset[newcols] = binary_encoded
   
train_df.head()

,survived,pclass,sex,age,sibsp,parch,fare,C,Q,S
0,0.0,3.0,0,1,0.0,0.0,1.0,0,0,1
1,0.0,1.0,0,1,0.0,0.0,3.0,1,0,0
2,0.0,2.0,0,1,0.0,0.0,2.0,1,0,0
3,0.0,2.0,0,2,0.0,0.0,1.0,0,0,1
4,0.0,3.0,0,1,0.0,0.0,1.0,0,0,1


In [208]:
X = train_df.drop("survived", axis=1) # Training & Validation data
Y = train_df["survived"] 

In [ ]:
# Random Forest
import Ipynb_importer
from hw5_Q2_spam import RandomForest,accuracy
random_forest,_ = accuracy(train_df,'survived', RandomForest, 30,20) 

In [211]:
y_pred = []
for i in range(len(test_df)):
    y_pred.appendrandom_forest.predict(test_df.iloc[i,:]))
y_pred
y_pred = np.array(y_pred)


In [212]:
# A code snippet to help you save your results into a kaggle accepted csv
import pandas as pd
import numpy as np

# Usage results_to_csv(clf.predict(X_test))
def results_to_csv(y_test):
    y_test = y_test.astype(int)
    df = pd.DataFrame({'Category': y_test})
    df.index += 1  # Ensures that the index starts at 1. 
    df.to_csv('submission.csv', index_label='Id')

In [213]:
results_to_csv(y_pred)

In [217]:
import Ipynb_importer
from hw5_Q2_spam import DecisionTree

dt=DecisionTree()
dt.train(train_df,'survived',3)
print(dt.root.status())
print(dt.root.left.status())
print(dt.root.right.status())
print(dt.root.left.left.status())
print(dt.root.left.right.status())
print(dt.root.right.left.status())
print(dt.root.right.right.status())
print(dt.root.left.left.left.status())
print(dt.root.left.left.right.status())
print(dt.root.left.right.left.status())
print(dt.root.left.right.right.status())
print(dt.root.right.left.left.status())
print(dt.root.right.left.right.status())
print(dt.root.right.right.left.status())
print(dt.root.right.right.right.status())

('sex', 1)
('pclass', 2.0)
('pclass', 3.0)
('age', 1)
('age', 1)
('S', 1)
('fare', 3.0)
1.0
1.0
1.0
0.0
1.0
1.0
1.0
0.0
